In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost.sklearn import XGBRegressor

In [2]:
from sklearn.model_selection import GridSearchCV

In [3]:
def read_clean_weather_data():
    sheet_name = ['2009','2010','2011','2012','2013','2014','2015','2016']
    weatherData = pd.DataFrame()
    for sheet in sheet_name:
        temp = pd.read_excel('data/WeatherData.xlsx',sheet_name=sheet)
        temp.Year=int(sheet)
        weatherData = pd.concat([weatherData,temp],ignore_index=True)

    weatherData.drop(weatherData.columns[-2:],axis=1,inplace=True)
    special_chars = ['!', '@', '#', '$', '%', '^', '&', '*', '(', ')', '_', '+', '-', '=', '{', '}', '[', ']', '|', '\\', ':', ';', '"', "'", '<', '>', '?', ',', '.', '/']
    weatherData = weatherData.replace(special_chars,'0')
    for col in weatherData.columns[3:]:
        weatherData[col] = weatherData[col].astype(float)
    
    dict_map= dict(zip(weatherData.Month.unique(),[1,2,3,4,5,6,7,8,9,10,11,12]))
    weatherData['Month'] = weatherData['Month'].map(dict_map)
    agg_func = {}
    for col in weatherData.columns[3:]:
        agg_func[col] = ['median']
    
    weatherData = weatherData.groupby(['Year','Month']).agg(agg_func).reset_index()
    weatherData = weatherData.droplevel(level=1,axis=1)
    
    return weatherData

In [4]:
def read_Events_HolidaysData():
    Events_HolidaysData = pd.read_excel("data/Events_HolidaysData.xlsx")
    Events_HolidaysData['Year'] = Events_HolidaysData['MonthDate'].dt.strftime('%Y')
    Events_HolidaysData['Month'] = Events_HolidaysData['MonthDate'].dt.strftime('%m')
    Events_HolidaysData['day'] = Events_HolidaysData['MonthDate'].dt.strftime('%d')
    Events_HolidaysData['Year'] = Events_HolidaysData['Year'].astype(int)
    Events_HolidaysData['Month'] = Events_HolidaysData['Month'].astype(int)
    Events_HolidaysData['day'] = Events_HolidaysData['day'].astype(int)
    Events_HolidaysData = Events_HolidaysData.groupby(['Year','Month']).agg({'Event':['count'],'DayCategory':['count']}).reset_index()
    Events_HolidaysData =  Events_HolidaysData.droplevel(level=1,axis=1)
    return Events_HolidaysData
        

In [5]:
def read_clean_macro_economic():
    macro_economic = pd.read_excel("Data/macro_economic.xlsx")
    macro_economic['Year']  = macro_economic['Year-Month'].str.split("-", expand = True)[0].str.strip()
    macro_economic['Month']  = macro_economic['Year-Month'].str.split("-", expand = True)[1].str.strip()
    dict_map= dict(zip(macro_economic.Month.unique(),[1,2,3,4,5,6,7,8,9,10,11,12]))
    macro_economic['Month'] = macro_economic['Month'].map(dict_map)
    macro_economic['Year'] = macro_economic['Year'].astype(int)
    return macro_economic
    

##### train,weatherData_month,Events_HolidaysData_count,macro_economic

In [6]:
def createdata(flag):
    final_data = ""
    y_train =""
    if flag == "train":
        final_data = pd.read_csv("Data/train.csv")
        final_data = final_data[~(final_data['Sales(In ThousandDollars)'].isnull())]
        y_train = final_data['Sales(In ThousandDollars)']
        final_data.drop(['Sales(In ThousandDollars)'],inplace=True,axis=1)
    if flag == "test":
        final_data = pd.read_csv("Data/submission.csv")
        final_data = final_data[final_data.columns[:3]]

    weatherData = read_clean_weather_data()
    HolidaysData = read_Events_HolidaysData()
    macro_economic = read_clean_macro_economic()
    final_data =  pd.merge(final_data,weatherData,on=['Year','Month'],how='inner')
    final_data = pd.merge(final_data,HolidaysData,on=['Year','Month'],how='left')
    final_data = pd.merge(final_data,macro_economic,on=['Year','Month'],how='left')
    final_data.drop(["AdvertisingExpenses (in Thousand Dollars)"],axis=1,inplace=True)
    final_data.drop(['PartyInPower'],inplace=True,axis=1)
    final_data.drop(['Year-Month'],axis=1,inplace=True)
    final_data['Event'] = final_data['Event'].replace(np.nan,0)
    final_data['DayCategory'] = final_data['DayCategory'].replace(np.nan,0)
    if flag == "train":
        return final_data,y_train
    return final_data

In [7]:
X_train,y_train = createdata("train")

In [8]:
test_data = createdata("test")

In [9]:
class Preprocessor(BaseEstimator,TransformerMixin):
    # Train our custom preprocessors
    def fit(self,X,y=None):
        
        self.scaler = StandardScaler()
        self.scaler.fit(X[X.columns[3:]])

        self.onehot = OneHotEncoder(handle_unknown = 'ignore')
        self.onehot.fit(X[['ProductCategory']])

        return self

    
    # Apply our custom preprocessors 
    def transform(self,X):
        #Apply Simple imputer
        onehot_cols = self.onehot.transform(X[['ProductCategory']])

        # Copy the df
        transformed_df = X.copy()
        
        # Apply transformed columns 
        transformed_df[transformed_df.columns[3:]] =  self.scaler.transform(transformed_df[transformed_df.columns[3:]])
        transformed_df = transformed_df.drop('ProductCategory',axis=1)
        transformed_df[self.onehot.get_feature_names_out()] = onehot_cols.toarray().astype(int)
        print(transformed_df.shape)
        return transformed_df

In [ ]:
from sklearn.model_selection import train_test_split

In [12]:
pipelines = {
    'ridge': make_pipeline(Preprocessor(), Ridge()), 
    'rf': make_pipeline(Preprocessor(), RandomForestRegressor()), 
    'gb': make_pipeline(Preprocessor(), GradientBoostingRegressor()), 
    'xg': make_pipeline(Preprocessor(), XGBRegressor()), 
}

In [13]:
grid = {
    'ridge':{'ridge__alpha':[0.05, 0.25, 0.5, 1.0]}, 
    'rf':{
        'randomforestregressor__n_estimators':[100,200,300], 
        'randomforestregressor__max_depth':[5,6,7,None]
    },
    'gb':{
        'gradientboostingregressor__n_estimators':[100,200,300], 
        'gradientboostingregressor__max_depth':[5,6,7, None]
    },
    'xg':{
       'xgbregressor__n_estimators':[100,200,300], 
       'xgbregressor__max_depth':[5,6,7,None]
    }
}

In [14]:
fit_models = {}
for algo, pipeline in pipelines.items(): 
    try: 
        print(algo)
        model = GridSearchCV(pipeline, grid[algo], n_jobs=-1, cv=10, scoring='r2',error_score='raise')
        model.fit(X_train,y_train)
        fit_models[algo] = model 
    except Exception as e: 
        print(f'Model {algo} had an error {e}')

ridge
(170, 40)
rf
(170, 40)
gb
(170, 40)
xg
(170, 40)


In [15]:
fit_models

{'ridge': GridSearchCV(cv=10, error_score='raise',
              estimator=Pipeline(steps=[('preprocessor', Preprocessor()),
                                        ('ridge', Ridge())]),
              n_jobs=-1, param_grid={'ridge__alpha': [0.05, 0.25, 0.5, 1.0]},
              scoring='r2'),
 'rf': GridSearchCV(cv=10, error_score='raise',
              estimator=Pipeline(steps=[('preprocessor', Preprocessor()),
                                        ('randomforestregressor',
                                         RandomForestRegressor())]),
              n_jobs=-1,
              param_grid={'randomforestregressor__max_depth': [5, 6, 7, None],
                          'randomforestregressor__n_estimators': [100, 200,
                                                                  300]},
              scoring='r2'),
 'gb': GridSearchCV(cv=10, error_score='raise',
              estimator=Pipeline(steps=[('preprocessor', Preprocessor()),
                                        ('gra

In [16]:
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [17]:
test_data.head()

,Year,Month,ProductCategory,Temp high (°C),Temp avg (°C),Temp low (°C),Dew Point high (°C),Dew Point avg (°C),Dew Point low (°C),Humidity (%) high,...,"Finance Rate on Personal Loans at Commercial Banks, 24 Month Loan",Earnings or wages in dollars per hour,Cotton Monthly Price - US cents per Pound(lbs),Change(in%),Average upland planted(million acres),Average upland harvested(million acres),yieldperharvested acre,Production (in 480-lb netweright in million bales),Mill use (in 480-lb netweright in million bales),Exports
0,2014,1,WomenClothing,1.0,-2.5,-6.5,-6.5,-10.5,-16.0,75.0,...,10.22,24.35,90.96,3.97,10.206,7.465,807,12.551,3.58,9.75
1,2014,1,MenClothing,1.0,-2.5,-6.5,-6.5,-10.5,-16.0,75.0,...,10.22,24.35,90.96,3.97,10.206,7.465,807,12.551,3.58,9.75
2,2014,1,OtherClothing,1.0,-2.5,-6.5,-6.5,-10.5,-16.0,75.0,...,10.22,24.35,90.96,3.97,10.206,7.465,807,12.551,3.58,9.75
3,2014,2,WomenClothing,2.0,-0.5,-4.0,-3.5,-7.5,-11.5,78.5,...,10.09,24.58,94.05,3.40,10.206,7.465,807,12.551,3.58,9.75
4,2014,2,MenClothing,2.0,-0.5,-4.0,-3.5,-7.5,-11.5,78.5,...,10.09,24.58,94.05,3.40,10.206,7.465,807,12.551,3.58,9.75


In [19]:
for algo,model in fit_models.items():
    yhat = model.predict(X_train)
    r2 = r2_score(y_train,yhat)
    mae = mean_absolute_error(y_train,yhat)
    print(f'{algo}-- R2 Score:{r2}, MAE Score:{mae}')

(170, 40)
ridge-- R2 Score:0.9339889662032739, MAE Score:217.33045400615288
(170, 40)
rf-- R2 Score:0.9929730593511259, MAE Score:61.88733721580685
(170, 40)
gb-- R2 Score:0.9999929066791663, MAE Score:2.2628484737392864
(170, 40)
xg-- R2 Score:0.9999999723700853, MAE Score:0.1458709716796875


In [20]:
submssion = fit_models['gb'].predict(test_data)

(36, 40)


In [26]:
submssion = pd.DataFrame(submssion).reset_index()

In [27]:
submssion["index"] = submssion["index"] +1 

In [29]:
submssion.columns = ['Year','Sales(In ThousandDollars)']

In [30]:
submssion

,Year,Sales(In ThousandDollars)
0,1,2508.729980
1,2,610.836058
2,3,1177.300004
3,4,2942.350765
4,5,640.153074
5,6,1201.106113
6,7,3666.095206
7,8,673.629838
8,9,1258.704837
9,10,3723.891614
